In [47]:
import requests, json
from urllib.parse import quote
import folium
import pandas as pd
import numpy as np

In [48]:
filename = 'data/seoul_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [49]:
area = pd.read_csv('data/서울공원요약.csv')
area.head()

,공원명,면적,주소,경도,위도,분류,크기
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,15
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,3
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,15
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,중형,7
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,대형,15


In [50]:
area['자치구'] = area.주소.apply(lambda x: ''.join(x.split()[1]))
area.head()

,공원명,면적,주소,경도,위도,분류,크기,자치구
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,15,중구
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,3,강동구
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,15,과천시
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,중형,7,성동구
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,대형,15,마포구


In [51]:
area.set_index('자치구', inplace=True)
area.head()

,공원명,면적,주소,경도,위도,분류,크기
자치구,,,,,,,
중구,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,15
강동구,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,3
과천시,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,15
성동구,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,중형,7
마포구,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,대형,15


In [52]:
area.drop(['과천시','공릉2동'],inplace=True)

In [53]:
map = folium.Map([37.4,127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=area.면적,
    columns=[area.index, area.면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map)
map

In [54]:
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [55]:
gu_dict = get_text_location(geo_data)

In [56]:
map = folium.Map([37.4,127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=area.면적,
    columns=[area.index, area.면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(map)
map
for gu_name in area.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
map